<a href="https://colab.research.google.com/github/dmalch/genealogy-research/blob/main/GenealogyResearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Шаг 1

Открыть таблицы с индексированными данными из метрических книг.

In [ ]:
import pandas as pd
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

# Open the spreadsheet
sh = gc.open('Кириллово общий индекс')

# Select the worksheet
births = sh.worksheet('Рождения')

# Get all values from the worksheet
birthsData = births.get_all_values()

# Convert data to pandas DataFrame
births_df = pd.DataFrame(birthsData[1:], columns=birthsData[0])

# Now you can work with the DataFrame 'df'
print(births_df.head())


  Номер муж Номер жен        Дата           Место                      Имя  \
0         1            01.01.1822     с. Кирилово      Ефим Спиридонов [1]   
1         2            01.01.1822     с. Кирилово      Василий Сидоров [5]   
2                   1  01.01.1822     с. Кирилово  Настасия Харитонова [6]   
3         3            02.01.1822  д. Гоголев Бор       Василий Иванов [7]   
4         4            02.01.1822  д. Гоголев Бор       Исаия Филиппов [8]   

       Имя в документе                    Отец  Отец в документе Мать  \
0      Ефим Спиридонов      Спиридон Фомин [2]    Спиридон Фомин        
1      Василий Сидоров    Сидор Ермолаев [235]    Сидор Ермолаев        
2  Настасия Харитонова  Харитон Панфенов [236]  Харитон Панфенов        
3       Василий Иванов        Иван Иванов [69]       Иван Иванов        
4        Исаия Филипов   Василий Исаевич [238]   Василий Исаевич        

  Мать в документе         Воспреемник 1 Воспреемник 1 в документе  \
0                     

# Шаг 2

Подготовка вспомогательных dataframes.

In [ ]:
# Extracting a subset of columns
children_df = births_df[['Имя в документе', 'Место']]
fathers_df = births_df[['Отец в документе', 'Место']]
mothers_df = births_df[['Мать в документе', 'Место']]
godparents1_df = births_df[['Воспреемник 1 в документе', 'Место']]
godparents2_df = births_df[['Воспреемник 2 в документе', 'Место']]

children_df.columns = ['Name', 'Place']
fathers_df.columns = ['Name', 'Place']
mothers_df.columns = ['Name', 'Place']
godparents1_df.columns = ['Name', 'Place']
godparents2_df.columns = ['Name', 'Place']

combined_df = pd.concat([children_df, fathers_df, mothers_df, godparents1_df, godparents2_df], ignore_index=True)

# Displaying the new DataFrames
# print(children_df.head())
# print(fathers_df.head())
# print(mothers_df.head())
# print(godparent1_df.head())
# print(godparent2_df.head())
# print(combined_df.head())
# print(combined_df.tail())
# print(combined_df.info())
# print(combined_df.describe())

In [ ]:
import graphviz
from IPython.display import Image

# Visualization using Graphviz
def visualize_family_tree():
  dot = graphviz.Digraph(comment='Family Tree')
  dot.attr('node', shape='box')

  for index, row in births_df.iterrows():

    # row=births_df.iloc[0]

    person = row['Имя']
    if person == '':
      continue
    dot.node(person, f'{person}\n{row["Дата"]}')
    father = row['Отец']
    dot.node(father)
    dot.edge(father, person, color='blue')
    mother = row['Мать']
    if mother != '':
      dot.node(mother)
      dot.edge(mother, person)
    godparent1 = row['Воспреемник 1']
    if godparent1 != '':
      dot.node(godparent1)
      dot.edge(godparent1, person, style='dashed')
    godparent2 = row['Воспреемник 2']
    if godparent2 != '':
      dot.node(godparent2)
      dot.edge(godparent2, person, style='dashed')

  dot = dot.unflatten(stagger=5)
  return dot

visualize_family_tree()